<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0"> </div>
    <div style="float: left; margin-left: 10px;"> 
        <h1>Natural Language Processing For Everyone</h1>
        <h1>Topic Modeling</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
        @bgoncalves, @data4sci</p></div>
</div>

In [1]:
import string
from collections import Counter
from pprint import pprint
import gzip

import matplotlib
import matplotlib.pyplot as plt 

import numpy as np
from numpy.linalg import svd
from numpy import diag

import watermark

%matplotlib inline
%load_ext watermark

List out the versions of all loaded libraries

In [2]:
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.11.8
IPython version      : 8.23.0

Compiler    : Clang 15.0.0 (clang-1500.3.9.4)
OS          : Darwin
Release     : 23.4.0
Machine     : arm64
Processor   : arm
CPU cores   : 10
Architecture: 64bit

Git hash: 8ff76187caba06cac839853d5ba2f90ad7e7e58e

matplotlib: 3.8.3
watermark : 2.4.3
numpy     : 1.26.4



Set the default style

In [3]:
plt.style.use('d4sci.mplstyle')

## Preprocessing

Load a standard list of stopwords from a file

In [4]:
stopwords = set([word.lower().strip() for word in open("data/nltk_stopwords.txt", "rt").readlines()])

In [5]:
list(stopwords)[:10]

["needn't",
 'how',
 'being',
 'in',
 'down',
 'because',
 'o',
 'again',
 'into',
 "shan't"]

Define a function to extract the words from our input file. Here we have to be (slightly) more careful than in the previous lesson since we want to keep #tags and @mentions and remove stopwords.

In [6]:
def extract_words(text, stopwords):
    temp = text.split() # Split the text on whitespace
    text_words = []

    punctuation = set(string.punctuation)
    
    # Keep #tags and @mentions
    punctuation.remove("#")
    punctuation.remove("@")
    
    for word in temp:
        # Remove any punctuation characters present in the beginning of the word
        while len(word) > 0 and word[0] in punctuation:
            word = word[1:]

        # Remove any punctuation characters present in the end of the word
        while len(word) > 0 and word[-1] in punctuation:
            word = word[:-1]

        # Simple rule to eliminate (most) URLs "http://www.google.com"
        if len(word) > 0 and "/" not in word:
            # If it's not a stopword
            if word.lower() not in stopwords:
                # Append this word into our list of words.
                text_words.append(word.lower())
        
    return text_words

Process the data from the file. File was obtained from crowdflower (https://www.crowdflower.com/data-for-everyone/) and contains a list of tweets about Apple from 2014 and some information about their sentiment. For our purposes, we only care about the text of each tweet that is located in the last column of the csv file.

We treat each tweet as an individual document, and limit ourselves to only the first 200 tweets.

In [9]:
tweets = []
line_count = 0

for line in open("data/Apple-Twitter-Sentiment-DFE.csv", "rt"):
    fields = line.strip().split(',')
    
    line_count += 1
    
    # Skip the first line of the file which contains the header
    if line_count == 1:
        continue
    
    tweet = ",".join(fields[11:])
    
    if len(tweet) == 0:
        continue
    
    words = extract_words(tweet, stopwords)
    
    if len(words) > 0:
        tweets.append(words)
        
    if len(tweets) == 200:
        break

In [10]:
tweets[:5]

[['cat', 'chews', '@apple', 'cords', '#applesnob'],
 ['nobody', 'expects', 'spanish', 'inquisition', '#aapl'],
 ['#aapl:5', 'rocket', 'stocks', 'buy', 'december', 'gains', 'apple'],
 ['top', '3', '@apple', '#tablets', 'damn', 'right'],
 ['apple', 'inc', 'flash', 'crash', 'need', 'know', '#aapl']]

Define the function to calculate the Inverse Document Frequency for each word and the TFIDF matrix. These functions were introduced in the previous lesson.

In [11]:
def inv_doc_freq(corpus_words):
    number_docs = len(corpus_words)
    
    document_count = {}

    for document in corpus_words:
        word_set = set(document)

        for word in word_set:
            document_count[word] = document_count.get(word, 0) + 1
    
    IDF = {}
    
    for word in document_count:
        IDF[word] = np.log(number_docs/document_count[word])
        
    
    return IDF

def tf_idf(corpus_words):
    IDF = inv_doc_freq(corpus_words)
    
    TFIDF = []
    
    for document in corpus_words:
        TFIDF.append(Counter(document))
    
    for document in TFIDF:
        for word in document:
            document[word] = document[word]*IDF[word]
            
    return TFIDF

Get the TFIDF matrix for our corpus of tweets. We should note that while we call it a matrix, this is effectively a list of dictionaries, which we can consider to be a sparse representation of a matrix.

In [12]:
TFIDF = tf_idf(tweets)

In [13]:
TFIDF[:5]

[Counter({'cat': 5.298317366548036,
          'chews': 5.298317366548036,
          'cords': 5.298317366548036,
          '#applesnob': 5.298317366548036,
          '@apple': 0.2169130015635736}),
 Counter({'nobody': 5.298317366548036,
          'expects': 5.298317366548036,
          'spanish': 5.298317366548036,
          'inquisition': 5.298317366548036,
          '#aapl': 1.9310215365615626}),
 Counter({'#aapl:5': 5.298317366548036,
          'rocket': 5.298317366548036,
          'stocks': 5.298317366548036,
          'gains': 5.298317366548036,
          'december': 4.605170185988092,
          'buy': 3.506557897319982,
          'apple': 2.5902671654458267}),
 Counter({'top': 5.298317366548036,
          '#tablets': 5.298317366548036,
          '3': 4.605170185988092,
          'damn': 4.605170185988092,
          'right': 3.912023005428146,
          '@apple': 0.2169130015635736}),
 Counter({'know': 4.605170185988092,
          'flash': 4.199705077879927,
          'crash': 3.9

For convenience, extract the word_dict and word_list from the TFIDF matrix.

In [14]:
def build_vocabulary(TFIDF):
    words = set()
    
    for document in TFIDF:
        words |= set(document.keys())
    
    word_list = list(words)
    word_dict = dict(zip(word_list, range(len(word_list))))
    
    return word_dict, word_list

In [15]:
word_dict, word_list = build_vocabulary(TFIDF)

In [16]:
vocabulary_size = len(word_dict)
print("We have", vocabulary_size, "words in our vocabulary")

We have 927 words in our vocabulary


In [18]:
word_list[:10]

['century',
 'contacts',
 '@farmville2',
 'screwed',
 'people',
 'mt',
 'get',
 'wtf',
 'gave',
 'draining']

Now use the TFIDF matrix and our vocabulary to generate the Term Document matrix. This is just a matter of rearranging the values in our (sparse) TFIDF matrix into the full TD matrix.

In [19]:
def term_document_matrix(TFIDF, word_list, word_dict):
    vocabulary_size = len(word_dict)
    number_documents = len(TFIDF)
    
    TDM = np.zeros((vocabulary_size, number_documents))
    
    for doc in range(number_documents):
        document = TFIDF[doc]
        
        for word in document.keys():
            pos = word_dict[word]
            
            TDM[pos, doc] = document[word]
            
    return TDM

In [20]:
TDM = term_document_matrix(TFIDF, word_list, word_dict)
print("Our dataset has:\n%u unique words\n%u documents"%(TDM.shape))

Our dataset has:
927 unique words
200 documents


In [21]:
TDM.shape

(927, 200)

## Explicit Semantic Analysis

In ESA we use the TD matrix of our corpus as a knowledge base that we can use to look up related documents. To demonstrate how this works, let's create a new document:

In [22]:
new_tweet = ['#macbook', 'mini', 'rocket']

This is not an actual sentence since, for clarity, we are not including stopwords.

In the TD formulation, the values in the word (row) vector representa how much each document is "activated" by that specific word. Naturally, documents in which the word does not appear have weight zero. We can use these word vectors to find a vector representation of our new document where each value is our related it is to every other document in our knowledge base. You can think of this as the set of results one might obtain after a query in a search engine!

The **find_related** simply calcuates the vector corresponding to the new "document" and returns a list of the corresponding weights sorted in decreasing order:

In [23]:
def find_related_docs(tweet, TDM):
    new_vector = np.zeros(TDM.shape[1])
    
    for word in tweet:
        pos = word_dict[word]
        new_vector += TDM[pos, :]
        
    # Now the entries of new_vector tell us which documents are activated by this one.
    # Let's extract the list of documents sorted by activation
    doc_list = sorted(zip(range(TDM.shape[1]), new_vector), 
                      key=lambda x: x[1], reverse=True)
    
    return doc_list

In [24]:
related = find_related_docs(new_tweet, TDM)

In [25]:
related[:5]

[(166, 6.607650686531799),
 (167, 6.607650686531799),
 (190, 6.607650686531799),
 (191, 6.607650686531799),
 (2, 5.298317366548036)]

Let's take a look a the top 5 results:

In [26]:
for tweet_id, score in related[:5]:
    print(score, tweet_id, " ".join(tweets[tweet_id]))

6.607650686531799 166 ipad mini unboxing via @youtube @apple #ipadmini #ipad #macbook #macbookpro #startup #hipster #unboxing
6.607650686531799 167 ipad mini first time startup via @youtube @apple #ipadmini #ipad #macbook #macbookpro #startup #hipster #unbox
6.607650686531799 190 ipad mini unboxing via @youtube @apple #ipadmini #ipad #macbook #macbookpro #startup #hipster #unboxing
6.607650686531799 191 ipad mini first time startup via @youtube @apple #ipadmini #ipad #macbook #macbookpro #startup #hipster #unbox
5.298317366548036 2 #aapl:5 rocket stocks buy december gains apple


With such small documents, the results are easy to interpret. The first two result are due to mentions of **macbook** and **mini**, the third due to the presence of the word **rocket** but the remaining two results don't include any words in common with our original tweet, just words that appear in similar contexts.

In [27]:
def cosine_similarity(vec1, vec2):
    sim = np.dot(vec1, vec2)
    norm1 = np.sqrt(np.dot(vec1, vec1))
    norm2 = np.sqrt(np.dot(vec2, vec2))
    
    return sim/(norm1*norm2)

In [29]:
def find_similar_words(tweet, TDM):
    new_vector = np.zeros(TDM.shape[1])
    
    for word in tweet:
        pos = word_dict[word]
        new_vector += TDM[pos, :]
    
    sim = [cosine_similarity(new_vector, TDM[i, :]) for i in range(TDM.shape[0])]
    
    sim_words = sorted(zip(range(TDM.shape[0]), sim), 
                       key=lambda x: x[1], reverse=True)
    
    return sim_words

In [30]:
similar = find_similar_words(new_tweet, TDM)

In [31]:
new_tweet

['#macbook', 'mini', 'rocket']

In [32]:
for word, score in similar[:10]:
    print(word_list[word], score)

#macbook 0.842346298979266
#hipster 0.8273747381258668
@youtube 0.8273747381258668
#startup 0.8273747381258668
mini 0.8227410508913036
#ipadmini 0.7963450852368475
#macbookpro 0.7800563838816592
ipad 0.7617102018231607
via 0.7055875474518816
#ipad 0.6502130393344844


This approach also returns words that are not included in the document but that are used in a similar context, such as #hipster. One might use this extra words (above some fine tuned threshold of similarity) to expand our query and retrieve further related documents.

## Latent Semantic Analysis

LSA analysis, sometimes refered to as Latent Semantic Indexing, relies on the well known Singular Value Decomposition. Singular Values (and Singular Vectors) can be thought of as being a generalization of the more common eigenvalues (and eigenvectors) for the case of non-square matrices. 

The first step in any LSA analysis is to apply SVD to the Term-Document matrix:

In [33]:
u, sigma, vt = svd(TDM)

The resulting matrices have the expected shapes, except the singular value matrix, **sigma** that is simply a vector. This is an optimization of the numpy function meant to save memory.

In [34]:
m, n = TDM.shape
print(u.shape, sigma.shape, vt.shape)

(927, 927) (200,) (200, 200)


In [35]:
print(m, n)

927 200


We can now reduce the internal dimention of these matrices to, say, 10. 

In [36]:
k = 10

# Drop the extraneous dimensions in the other two matrices.
uk = u[:, :k]
vtk = vt[:k, :]

# Convert the vector of singular values into a diagonal matrix
sigma_k = sigma[:k]
Sk = diag(sigma_k)

In this way, we preserve the most significant (higher sigma values) latent dimensions of the dataset while discarding the rest. The idea is simple, SVD projects the original dataset into an internal space which is a linear combination of the original one. Each dimension in this space can be thought of as an underlying topic in the data. By preserving only the most dominant topics in effect we are simplifying the data and, potentially, removing noise. 

In [37]:
print(uk.shape, Sk.shape, vtk.shape)

(927, 10) (10, 10) (10, 200)


There are no easy ways to determine the number of topics to choose, but a good rule of thumb is to identify kinks in the values of sigma

Text(0, 0.5, '$\\sigma_i$')

In [40]:
plt.plot(range(200), sigma, '*')
plt.xlabel(r'$i$')
plt.ylabel(r'$\sigma_i$')

We can also explore the way in which the topics are defined by considering the words for which they have stronger weights:

In [41]:
# Sort the list of words by the weight they have in a specific topic
def top_words(vector, word_list):
    doc_list = sorted(zip(word_list, vector), key=lambda x:x[1], reverse=True)
    
    return doc_list

For example, for topic 3, we have strong mentions of AIDS day and the Produc (RED) iPhone launch:

In [42]:
topic_words = top_words(uk[:, 3], word_list)
for word, weight in topic_words[:10]:
    print(word, weight)

day 0.287464880143303
aids 0.28262164097538983
world 0.2692253682039425
red 0.26475561492615873
logo 0.22903406070460594
#apple 0.1850434730267795
apple 0.13868223906604202
dec 0.12638896410764333
abl 0.12638896410764333
launches 0.12638896410764333


On the other hand, if we were trying to map a specific sentence or document into the most relevant topic we would simply have to project it into the singular space. First we define the vector in the "regular" word space:

In [43]:
document = ['ipad', 'mini', 'price']

doc_vector = np.zeros(vocabulary_size)
for word in document:
    doc_vector[word_dict[word]] += 1

and now we project it into the singular space:

In [44]:
doc_singular = 1/sigma_k*np.dot(uk.T, doc_vector)
pprint(doc_singular)

array([-1.62243400e-04,  1.44900491e-02, -1.09973213e-03,  4.30257980e-05,
        9.69106965e-04, -4.54646796e-03,  7.58613138e-04, -2.40772326e-03,
        3.82599561e-03, -1.95924079e-03])


From where we see that the most relevant topic is topic number 1:

In [45]:
topic = np.argmax(doc_singular)

print(topic)

topic_words = top_words(uk[:,topic], word_list)
for word, weight in topic_words[:10]:
    print(word, weight)

1
#macbook 0.3072355938213901
#hipster 0.3054014569972668
@youtube 0.3054014569972668
#startup 0.3054014569972668
#macbookpro 0.2980960628908025
via 0.29187957210987786
mini 0.20949666233803638
ipad 0.20381766001212492
#ipadmini 0.19891025664321704
startup 0.1932748800761491


Which indeed seems relevant.

## Non-Negative Matrix Factorization

The idea behind applying NMF to a Term-Document Matrix is similar to the goals behind using SVD to perform Latent Semantic Analysis. NMF is a general matrix factorization algorithm and, as such, the factor matrices it finds can also be thought of as internal or latent dimensions. 

Similarly to the case of SVD, we can perform a partial reconstruction of the original matrix by reducing the internal dimensions. Naturally, as the number of latent dimensions increases, the reconstruction error decreases since we are removing less and less information:

In [49]:
def NMF(V, k):
    n, m = V.shape

    W = np.random.random((n, k))
    H = np.random.random((k, m))

    error = 1
    err = 1

    while err > 1e-3:
        old_error = error

        hn = np.dot(W.T, V)
        hd = np.dot(W.T, np.dot(W, H))

        H = H*np.divide(hn, hd)

        wn = np.dot(V, H.T)
        wd = np.dot(W, np.dot(H, H.T))

        W = W*np.divide(wn, wd)

        error = np.sum(np.power(V-np.dot(W, H), 2.0))
        err = np.abs(old_error-error)

    return W, H, error

Let's see how the error changes with the number of components.

In [50]:
%%time
error = []

for comps in range(2, 25+1):
    W, H, err = NMF(TDM, comps)
    print(comps, err)
    error.append(err)

2 32018.606987559608
3 31373.947061933963
4 30774.185434748553
5 30274.10984834639
6 29792.906073379545
7 29346.023579773017
8 28894.408527905864
9 28464.118664657806
10 28051.30671513068
11 27601.68423314626
12 27263.63106789977
13 26824.53747594752
14 26394.024274316635
15 26044.995261347343
16 25635.787916981957
17 25270.977370004388
18 24868.84916221801
19 24475.578782559285
20 24171.849142886913
21 23755.28462512858


/var/folders/l6/0wz4_c0j4yl29mqzds3h7gwm0000gn/T/ipykernel_47730/811614206.py:21: RuntimeWarning: overflow encountered in divide
  W = W*np.divide(wn, wd)
/var/folders/l6/0wz4_c0j4yl29mqzds3h7gwm0000gn/T/ipykernel_47730/811614206.py:21: RuntimeWarning: invalid value encountered in multiply
  W = W*np.divide(wn, wd)


22 nan
23 23149.518036968322
24 22835.959302926563
25 22450.966827683187
CPU times: user 4min 59s, sys: 10.5 s, total: 5min 9s
Wall time: 34.7 s


Any __nan__ values found above are due to numerical instabilities. In practice you should use a well tested numerical library such as __sklearn__.

Text(0.5, 1.0, 'Non-Negative Matrix Factorization')

In [51]:
plt.plot(range(2, 25+1), error)
plt.xlabel('Number of Components')
plt.ylabel('Reconstruction Error')
plt.title('Non-Negative Matrix Factorization')

For the remainder of this section, lets fix the number of so that we can use these results to illustrate the power of NMF.

In [52]:
W, H, err = NMF(TDM, 10)

When we apply NMF to a Term-Document matrix, as defined above, the resulting factors prove useful for topic detection. In particular, the W matrix provides us with the definition of each topic as a weighted distribution over all the words in the corpus:

Let's look at a couple of examples:

In [54]:
topic_words = top_words(W[:, 1], word_list)

for word, weight in topic_words[:10]:
    print(word, weight)

users 26.28980561737975
#cybermonday 13.393591262687776
21.6 13.144902808689874
@android 13.144902808689874
online 13.144902808689874
6.6 13.144902808689874
dominating 13.144902808689874
#business 13.144902808689874
#sales 13.144902808689874
account 13.144902808689874


In [55]:
topic_words = top_words(W[:, 2], word_list)

for word, weight in topic_words[:10]:
    print(word, weight)

today 13.880605351852404
#aapl 12.287157498765541
stock 11.854840836854502
trading 11.388175949077956
behind 9.833567899075893
apple's 9.650604548023855
customer 9.443658115170473
best 8.466157763119561
price 8.363322959995038
yesterday 7.789381569056665


<center>
     <img src="data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</center>